## **Compute**

A Jupyter notebook talking about computing a synthetic model in PHOEBE. This roughly follows the given tutorial provided at https://phoebe-project.org/docs/2.4/.

### Setup

Let's quickly install PHOEBE (if needed), load it and other librarys, set up the logger and load the default binary Bundle.

In [ ]:
# !pip install phoebe

In [1]:
import phoebe as phb
import numpy as np
import matplotlib.pyplot as plt

PHOEBE: passband "Bolometric:900-40000" has a newer version available.  Run phoebe.list_passband_online_history("Bolometric:900-40000") to get a list of available changes and phoebe.update_passband("Bolometric:900-40000") or phoebe.update_all_passbands() to update.
PHOEBE: passband "Johnson:V" has a newer version available.  Run phoebe.list_passband_online_history("Johnson:V") to get a list of available changes and phoebe.update_passband("Johnson:V") or phoebe.update_all_passbands() to update.


In [ ]:
logger = phb.logger()
bSystem = phb.default_binary()

: 

Let's also attach some dummy datasets to get things running.

In [ ]:
bSystem.add_dataset(
    'orb',
    compute_times = phb.linspace(0, 10, 10),
    dataset = 'orb01'
)

bSystem.add_dataset(
    'lc',
    compute_times = phb.linspace(0, 1, 101),
    dataset = 'lc01'
)

### Default Compute Options

Any default Bundle has a set of default compute options to run the backend for PHOEBE. In most cases, you can just edit the options in the default set.

In [ ]:
print(bSystem.computes)

In [ ]:
print(bSystem['compute'])

The above lists all the options for the default compute, and can be changed by setting the parameter to a specified number/option. Reminder: You can use `bSystem.get_limits` and `bSystem.get_choices` to see what is valid.

### Adding Compute Options

In other cases, you can manually add additional sets of compute options. This can be done using `bSystem.add_compute`.

In this example, we want to add two sets of compute options to work with. These will be labelled as 'preview', which runs a quick model at the expense of accuracy, and 'detailed', which gives an accurate model at the expense of compute time.

In [ ]:
bSystem.add_compute(
    'phoebe', # Compute model to use, which this just loads the default one
    compute = 'preview', # Label for our compute option, which is 'preview'
    irrad_method = 'none' # Changes the irrad method (which can be seen in the list of options in the previous code block) to none
)

bSystem.add_compute(
    'phoebe', # Compute model to use, which this just loads the default one
    compute = 'detailed', # Label for our compute option, which is 'detailed'
    irrad_method = 'wilson' # Changes the irrad method (which can be seen in the list of options in the previous code block) to wilson
)

In [ ]:
print(bSystem['compute@preview'])
print('\nIrradiation Method: {}'.format(bSystem['compute@preview@irrad_method'].get_value()))

In [ ]:
print(bSystem['compute@detailed'])
print('\nIrradiation Method: {}'.format(bSystem['compute@detailed@irrad_method'].get_value()))

### Editing Compute Options

Most of the parameters in the compute options are specific to the backend being used. We've only used the PHOEBE 2.4 backend, but others exist.

Different datasets will have their own specific compute options. The ones for light curves is at https://phoebe-project.org/docs/2.4/tutorials/LC.

### Enabling/Disabling Datasets

By default, synthetic models will be created for all datasets in the Bundle when `bSystem.run_compute` is called. Datasets can be disabled so that `run_compute` ignores it, which is controlled by the parameter 'enabled' in the dataset.

The 'enabled' parameter is separate for each compute option. Hence, you can set the dataset to be enabled/disabled for specific datasets. You can use the `bSystem.set_value_all` to enable/disable the dataset for all compute options.

In [ ]:
print(bSystem['enabled@lc01'])

In [ ]:
bSystem['preview@enabled@lc01'] = False
bSystem['phoebe01@enabled@lc01'] = False
print(bSystem['enabled@lc01'])

In [ ]:
bSystem.set_value_all('enabled@lc01', True)
print(bSystem['enabled@lc01'])

PHOEBE can automatically disable datasets for specific compute options, if a compute option does not support that dataset type.

## Running Compute

`bSystem.run_compute` takes arguments for the compute tag as well as the model tag for the resulting synthetic model(s).

The compute tag doesn't need to be provideed if only 0 or 1 set of compute options exists in the Bundle. If 0 exist, then the default PHOEBE 2.4 options will be added and used. If 1 exists, it will assume that you are using those.
If there are more than 1, then the tag must be provided.

If a tag isn't provided for the model, one will be created under the name 'latest'. This can be overwritten without any errors, whilst naming it anything else will throw an error if the `overwrite=True` tag is NOT provided. 
It is best practice to provide a model tag.

In [ ]:
bSystem.run_compute(compute='preview')
print(bSystem.models)

Now let's run 3 different models with inclinations 90 deg, 85 deg and 80 deg. This will use the preview compute option and we will provide different model tags to store the results

In [ ]:
bSystem['incl@orbit'] = 90
bSystem.run_compute(compute='preview', model='incl_90')

bSystem['incl@orbit'] = 85
bSystem.run_compute(compute='preview', model='incl_85') 

bSystem['incl@orbit'] = 80
bSystem.run_compute(compute='preview', model='incl_80') 

In [ ]:
print(bSystem.models)

Models can be removed using `bSystem.remove_model` and specifying the model.

Synthetic data can be accessed via their dataset and model tags.

In [ ]:
incl_90 = bSystem['incl_90@primary']
print(incl_90['us'])

print(incl_90['us'].get_value())

## compute_times & compute_phases

### Overriding Computation Times

If `compute_times` is not empty, the provided value will be used to compute the model instead of those in the `times` parameter.

`compute_times` will override the values in `times` when computing the model. However, passing `times` as a keyword into `bSystem.run_compute` will override precidence over both. Hence the hierarchy is:
1. `bSystem.run_compute(times=[0, 0.5, 1])`
2. `bSystem.add_dataset('lc', compute_times=[0, 0.5, 1])`
3. `bSystem.add_dataset('lc', times=[0, 0.5, 1])`

### Phase-Time Conversion

We can alternatively provide `compute_phases` instead of `compute_times`. These parameters are linked via a constraint and needs to be inverted before using `compute_phases` (PHOEBE always uses time when computing synthetic models, so `compute_phases` is constrained by default).

In [ ]:
bSystem = phb.default_binary()
bSystem.add_dataset(
    'lc',
    times=phb.linspace(0, 10, 101),
    dataset='lc01'
)

print(bSystem['compute_phases@lc01@dataset'])

In [ ]:
bSystem.flip_constraint('compute_phases', solve_for='compute_times') # Flip the constraint so we can set compute_phases and have PHOEBE work out the times for us
bSystem['compute_phases'] = phb.linspace(0, 1, 11)

print(bSystem.filter(qualifier=['times', 'compute_times', 'compute_phases', 'compute_phases_t0'], context='dataset'))

As we can see, the C value beside `compute_times@lc01@dataset` tells us that `compute_times` is now constrained. 

Note: `phases` does **NOT** exist for observational data. You **HAVE** to work with `times` when setting `fluxes`. If the data provided is a light curve in phase, then you need to use `bSystem.to_time` or convert the phase manually before being added to the dataset.

## Interpolation & Residuals

### Interpolation

In a given dataset, we can interpolate to a specific result or set of results within it using `bSystem.interp_value`.


In [ ]:
bSystem.run_compute()

print(bSystem['fluxes@model'].get_value())

In [ ]:
print(bSystem['fluxes@model'].interp_value(times=1.0))

In [ ]:
print(bSystem['fluxes@model'].interp_value(times=phb.linspace(0,3,101)))

When interpolating in time, it will automatically interpolate in phase-space if the provided time is outside the range of the referenced time array.

### Residuals

Residuals can be calculated using `bSystem.compute_residuals`, which handles interpolation and compares the dependent variable between model and observation. 

A test case for a light curve is given below.

In [ ]:
# Setting up a dataset to generate some 'observational' data
bSystem.add_dataset( 
    'lc',
    times=phb.linspace(0, 10, 1000),
    dataset='lc01',
    overwrite=True
)

bSystem.run_compute(irrad_method='none')
bSystem['fluxes@dataset'] = bSystem['fluxes@model'].get_value() # Set observational data

bSystem.flip_constraint('compute_phases', 'compute_times') # Flip to phase-space
bSystem['teff@primary'] = 5950 # Change temp for primary
bSystem['compute_phases'] = phb.linspace(0, 1, 101)
bSystem.run_compute(irrad_method='none')

res = bSystem.calculate_residuals()
print(res)

In [ ]:
afig, mplfig = bSystem.plot(show=True)

Here we've only computed one cycle, whilst the 'observational' dataset extends further in time.

We can plot the residuals as well. For some reason, `bSystem.plot(y='residuals')` doesn't work so a normal matplotlib figure is created instead. `bSystem.plot(y='residuals')` would be able to calculate the residuals and plot it automatically if it worked.

In [ ]:
fig, ax = plt.subplots()
ax.plot(bSystem['times@dataset'].get_value(), res,'+', c='k')
plt.show()

## Running Multiple Computes

### Running Compute with Multiple Sets of Options

You can run multiple computes on **different** datasets with **different** options using `bSystem.run_compute` by providing a list to `compute`. Let's quickly set up everything.

In [ ]:
bSystem = phb.default_binary()

bSystem.add_dataset(
    'orb',
    compute_times=phb.linspace(0,10,10),
    dataset='orb01',
    component=['primary','secondary']
)
bSystem.add_dataset(
    'lc',
    times=phb.linspace(0, 10, 100),
    dataset='lc01'
)

bSystem.add_compute(
    'phoebe',
    compute='preview',
    irrad_method='none'
)
bSystem.add_compute(
    'phoebe',
    compute='detailed',
    irrad_method='wilson'
)

Let's make it so that the lightcurve dataset runs in 'detailed' mode while the orbital dataset runs in 'preview' mode.

In [ ]:
# Set up orb dataset
bSystem.set_value_all('enabled@orb01', False)
bSystem['preview@enabled@orb01'] = True

# Set up lc dataset
bSystem.set_value_all('enabled@lc01', False)
bSystem['detailed@enabled@lc01'] = True

# Run compute
bSystem.run_compute(compute=['detailed', 'preview'], model='multicompute')

### Running Compute with MPI

MPI (Message Passing Interface) is a method to get code to run in parallelisation. PHOEBE has it inbuilt into `run_compute` and `run_solver`. To check if MPI is enabled, you can simply check if `phb.mpi.enabled` returns true.

In [ ]:
print(phb.mpi.enabled)

To turn it on use `phb.mpi_on`, which you can also specify the number of processes MPI has access to. To turn it off use `phb.mpi_off`.

In [ ]:
phb.mpi_on()
print(phb.mpi.enabled)

You can check how MPI will behave by calling `phb.mpi.mode`. You can also check the rank and number of processes with `phb.mpi.myrank` and `phb.mpi.nprocs` respectively.

In [ ]:
print(phb.mpi.mode)
print(phb.mpi.myrank)
print(phb.mpi.nprocs)

PHOEBE will determine if the current script is running in an MPI enviroment (which you would need to do from the command line using MPI itself), which can be checked by calling `phb.mpi.within_mpirun`.

In [ ]:
print(phb.mpi.within_mpirun)

If it is `False`, i.e. you are running Python in a serialised way, then using `phb.run_compute` will spawn a number of threads equal to the number of `nprocs` allocated.